In [5]:
#test
import numpy as np
import plotly.graph_objects as go
from scipy.spatial import ConvexHull

In [6]:
def get_finger_orientation(finger_orientation_deg:list):
    finger_orientation_rad = []
    for angle in finger_orientation_deg:
        finger_orientation_rad.append(angle /180 * np.pi)
    
    return finger_orientation_rad



def calculate_workspace_volume(X, Y, Z):
    """Compute the convex hull volume for a set of points."""
    try:
        points = np.column_stack((X, Y, Z))  # Convert to (N, 3) array
        hull = ConvexHull(points)
        volume = hull.volume
    except:
        volume = 0

    return volume
    



In [7]:
#Plot Options
points = True
solid = False

#finger position
finger_orientation_deg = [43,135,245]
x_position = [0.2,0.2,-0,1]
y_position = [0,0.1, 0.05]
z_position = [0,0,0]

finger_orientation = get_finger_orientation(finger_orientation_deg)

colors = ['red', 'green', 'blue' , 'yellow', 'cyan', 'magenta','orange']


segment_length = 0.2  # Each segment length
num_angles = 30# Reduced resolution for movement visualization


joint_range_1_list = [np.linspace(-np.pi/2, 0, num_angles) ,np.linspace(-np.pi/2, 0, num_angles) ,np.linspace(-np.pi/2, 0, num_angles) ]
joint_range_2_list = [np.linspace( -np.pi/2,0, num_angles), np.linspace( -np.pi/2,0, num_angles),np.linspace( -np.pi/2,0, num_angles)]
#joint_range_3_list = [np.linspace(-np.pi/5, 0, num_angles),np.linspace(-np.pi/5, 0, num_angles),np.linspace(-np.pi/5, 0, num_angles)   ]


base_rotation_range_list = [np.linspace(-np.pi/4 + finger_orientation[0], np.pi/4+ finger_orientation[0], num_angles * 2) ,
                            np.linspace(-np.pi/4 + finger_orientation[1], np.pi/4+ finger_orientation[1], num_angles * 2),
                            np.linspace(-np.pi/4 + finger_orientation[2], np.pi/4+ finger_orientation[2], num_angles * 2)] # Base rotation around Z-axis




volume_text_list = []
fig = go.Figure()

for i, base_angle in enumerate(finger_orientation): 
    X, Y, Z = [], [], []

    base_rotation_range = base_rotation_range_list[i]
    for phi in base_rotation_range:  # Reduce computed points by stepping [::2]
        
        joint_range_1 = joint_range_1_list[i]
        joint_range_2 = joint_range_2_list[i]
        #joint_range_3 = joint_range_3_list[i]
        
        for theta1 in joint_range_1[::2]:
            for theta2 in joint_range_2[::2]:
               # for theta3 in joint_range_3[::2]:
                    # Apply spacing only to the first two fingers
                  
                    x_offset = x_position[i]
                    y_offset = y_position[i]
                    z_offset = z_position[i]
                
                    
                    x1 = segment_length * np.cos(base_angle) * np.cos(theta1) + x_offset
                    y1 = segment_length * np.sin(base_angle) * np.cos(theta1) + y_offset
                    z1 = segment_length * np.sin(theta1) + z_offset
                    
                    x2 = x1 + segment_length * np.cos(base_angle) * np.cos(theta1 + theta2)
                    y2 = y1 + segment_length * np.sin(base_angle) * np.cos(theta1 + theta2)
                    z2 = z1 + segment_length * np.sin(theta1 + theta2)
                    
                    #x3 = x2 + segment_length * np.cos(base_angle) * np.cos(theta1 + theta2 + theta3)
                    #y3 = y2 + segment_length * np.sin(base_angle) * np.cos(theta1 + theta2 + theta3)
                    #z3 = z2 + segment_length * np.sin(theta1 + theta2 + theta3)
                    
                    x3 = x2
                    y3 = y2
                    z3 = z2
                    
                    # Apply base rotation around Y-axis
                    x_final = (x3-x_offset) * np.cos(phi) - (y3 - y_offset) * np.sin(phi) +x_offset
                    y_final = (x3-x_offset) * np.sin(phi) + (y3 - y_offset) * np.cos(phi) + y_offset
                    z_final = z3
                    
                    
                    X.append(x_final)
                    Y.append(y_final)
                    Z.append(z_final)
    
    
                  
    volume = calculate_workspace_volume(X, Y, Z)
    volume_text = f"Finger {i+1} DOF Space Volume: {volume:.4f}"
    print(volume)
    volume_text_list.append(volume_text)

    
    if points == True:
        fig.add_trace(go.Scatter3d(
            x=X, y=Y, z=Z,
            mode='markers',
            marker=dict(size=3, opacity=0.5, color=colors[i]),
            name=f'Finger {i+1}'
        ))
    
    #solid
    if solid == True:
        fig.add_trace(go.Mesh3d(
        x=X, y=Y, z=Z,
        color=colors[i],  # Use a solid color for the finger
        opacity=0.8
        ))
        
        
        
        
for i, text in enumerate(volume_text_list):

    
       
    fig.add_annotation(
        x=0.2, y=0.3 - (0.07 * i),  # Coordinates of the annotation
        text=text,  # Text to display
        showarrow=False,  # Display an arrow pointing to the text
        arrowhead=2,  # Type of arrow
        font=dict(size=9, color="white"),  # Font styling
        bgcolor=f"{colors[i]}",  # Background color
        opacity=0.8
    )
       
        

# Save as an interactive HTML file
fig.write_html("range_of_motion.html")
print("Interactive 3D plot saved as gripper_plot.html")


# Show the interactive plot
fig.show()




0.04648862800098769
0.04648862800098772
0.04648862800098745
Interactive 3D plot saved as gripper_plot.html
